<a href="https://colab.research.google.com/github/kesaroid/EIP-3.0-Phase-2/blob/master/LSTM_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load Larger LSTM network and generate text
import sys
import numpy
import string
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd 'drive/My Drive/EIP'
!ls

/content/drive/My Drive/EIP
A4  Prisma-esque  wonderland.txt


In [0]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [0]:
clean_text = raw_text.translate(str.maketrans('', '', string.punctuation))

# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(clean_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [6]:
# summarize the loaded data
n_chars = len(clean_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  154861
Total Vocab:  39


In [7]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = clean_text[i:i + seq_length]
	seq_out = clean_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)


Total Patterns:  154761


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [9]:
# define the LSTM model
model = Sequential()
model.add(Dropout(0.1))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

W0820 19:07:52.089005 140436864870272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0820 19:07:52.126985 140436864870272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [10]:
# define the checkpoint
filepath="weights-best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=50, batch_size=128, callbacks=callbacks_list)

W0820 19:07:54.158783 140436864870272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0820 19:07:54.162826 140436864870272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0820 19:07:54.177446 140436864870272 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0820 19:07:54.196263 140436864870272 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backe

Epoch 1/50
154761/154761 [==============================] - 352s 2ms/step - loss: 2.8015

Epoch 00001: loss improved from inf to 2.80155, saving model to weights-best.hdf5
Epoch 2/50
154761/154761 [==============================] - 348s 2ms/step - loss: 2.6094

Epoch 00002: loss improved from 2.80155 to 2.60939, saving model to weights-best.hdf5
Epoch 3/50
154761/154761 [==============================] - 347s 2ms/step - loss: 2.4449

Epoch 00003: loss improved from 2.60939 to 2.44493, saving model to weights-best.hdf5
Epoch 4/50
154761/154761 [==============================] - 347s 2ms/step - loss: 2.3285

Epoch 00004: loss improved from 2.44493 to 2.32855, saving model to weights-best.hdf5
Epoch 5/50
154761/154761 [==============================] - 347s 2ms/step - loss: 2.2445

Epoch 00005: loss improved from 2.32855 to 2.24451, saving model to weights-best.hdf5
Epoch 6/50
154761/154761 [==============================] - 346s 2ms/step - loss: 2.1782

Epoch 00006: loss improved from 2.

In [13]:
# load the network weights
filename = "weights-best.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

Seed:
" ittle way forwards each time and a long way back and barking hoarsely
all the while till at last it  "


In [16]:
# generate characters
for i in range(501):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

ce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu anlce tuoeieu 
Done.
